# chapter 5. Regression

## Linear Regression - Boston Home Price Forecast

### Linear Regression Class - Ordinary Least Square

**Linear Regression Class**는 **예측값과 실제 값의 RSS를 최소화해 OLS 추정 방식으로 구현한 클래스**이다. <br>
Linear Regression Class는 fit() 메서드로 X, y 배열을 입력받으면 회귀 계수인 W를 coef_ 속성에 저장한다. 

In [ ]:
class sklearn.linear_model.LinearRegression(fit_intercept = True, normalize = False, copy_X = True, n_jobs = 1)

**Ordinary Least Squares 기반의 회귀 계수 계산**은 **입력 피처의 독립성에 많은 영향**을 받는다. <br>
피처간의 상관관계가 높은 경우 분산이 매우 커져서 오류에 민감해진다. -> 이 현상을 **다중 공선성 문제**라고 한다. <br>
일반적으로 상관관계가 높은 피처가 많은 경우 독립적인 중요한 피처만 남기고 제거하거나 규지를 적용한다. <br>
PCA를 통해 차원 축소를 수행하는 것도 고려해 볼 수 있다. 

### 회귀 평가 지표

**회귀의 평가를 위한 지표**는 실제 값과 회귀 예측값의 차이 값을 기반으로 한 지표가 중심이다. <br>
**실제값과 예측값의 차이를 그냥 더하면 +와 -가 섞여 오류가 상쇄**된다. <br>
이 때문에 **오류의 절댓값 평균이나 제곱, 또는 제곱한 뒤 다시 루트를 씌운 평균값을 구한다.** <br>
일반적으로 회귀의 성능을 평가하는 지표는 아래와 같다. 

<img src = 'image/Regression Assessment Indicators.jpg' alt = 'Regression Assessment Indicators' width='600' height='500'>

**사이킷런의 API 및 cross)val_score나 GridSearchCV에서 평가 시 사용되는 scoring 파라미터의 적용값**

<img src = 'image/Scoring Value.jpg' alt = 'Scoring Value' width='600' height='400'>

RMSE를 구하기 위해서는 MSE를 위한 metrics.mean_squared_error() 함수를 그대로 사용하되, squared 파라미터를 False로 지정해 사용한다. <br>
즉 MSE는 사이킷런에서 mean_squared_error(실제값, 예측값, squared = True)이며 RMSE는 mean_squared_error(실제값, 예측값, squared = False)를 이용해서 구한다. 

Scoring 함수에 'neg_mean_absolute_error'를 적용해 음수값을 반환하는 이유는 사이킷런의 Scoring 함수가 score값이 클수록 좋은 평가 결과로 자동 평가하기 때문이다. <br>
실제값과 예측값의 오류 차이를 기반으로 하는 회귀 평가 지표의 경우 값이 커지면 오히려 나쁜 모델이라는 의미이므로 이를 사이킷런의 **Scoring 함수에 일반적으로 반영하려면 보정이 필요**하다. <br>
따라서 -1을 원래의 평가 지표 값에 곱해서 음수를 만들어 작은 오류 값이 더 큰 숫자로 인식하게 한다. 

### Linear Regression을 이용해 보스턴 주택 가격 회귀 구현

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.datasets import load_boston
import warnings
warnings.filterwarnings('ignore') # 사이킷런 1.2부터는 보스턴 주택가격 데이터가 없어진다는 warning 메시지 출력 제거
%matplotlib inline

# boston 데이타셋 로드
boston = load_boston()

# boston 데이타셋 DataFrame 변환 
bostonDF = pd.DataFrame(boston.data , columns = boston.feature_names)

# boston dataset의 target array는 주택 가격이다. 이를 PRICE 컬럼으로 DataFrame에 추가한다. 
bostonDF['PRICE'] = boston.target
print('Boston 데이타셋 크기 :',bostonDF.shape)
bostonDF.head()

In [ ]:
bostonDF.head(20)

In [ ]:
# 2개의 행과 4개의 열을 가진 subplots를 이용. axs는 4x2개의 ax를 가진다.
fig, axs = plt.subplots(figsize = (16,8) , ncols = 4 , nrows = 2)
lm_features = ['RM','ZN','INDUS','NOX','AGE','PTRATIO','LSTAT','RAD']
for i , feature in enumerate(lm_features):
    row = int(i / 4)
    col = i % 4
    # 시본의 regplot을 이용해 산점도와 선형 회귀 직선을 함께 표현
    sns.regplot(x=feature , y = 'PRICE',data = bostonDF , ax = axs[row][col])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score

y_target = bostonDF['PRICE']
X_data = bostonDF.drop(['PRICE'],axis = 1,inplace = False)

X_train , X_test , y_train , y_test = train_test_split(X_data , y_target ,test_size = 0.3, random_state = 156)

# Linear Regression OLS로 학습 / 예측 / 평가 수행
lr = LinearRegression()
lr.fit(X_train ,y_train )
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_test, y_preds)))

In [ ]:
print('절편 값:',lr.intercept_)
print('회귀 계수값:', np.round(lr.coef_, 1))

In [ ]:
# 회귀 계수를 큰 값 순으로 정렬하기 위해 Series로 생성하다. index가 컬럼명에 유의
coeff = pd.Series(data = np.round(lr.coef_, 1), index = X_data.columns)
coeff.sort_values(ascending = False)

In [ ]:
from sklearn.model_selection import cross_val_score

y_target = bostonDF['PRICE']
X_data = bostonDF.drop(['PRICE'],axis = 1,inplace = False)
lr = LinearRegression() 

# cross_val_score( )로 5 Fold 셋으로 MSE 를 구한 뒤 이를 기반으로 다시 RMSE를 구한다. 
neg_mse_scores = cross_val_score(lr, X_data, y_target, scoring="neg_mean_squared_error", cv = 5)
rmse_scores  = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

# cross_val_score(scoring = "neg_mean_squared_error")로 반환된 값은 모두 음수 
print(' 5 folds 의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print(' 5 folds 의 개별 RMSE scores : ', np.round(rmse_scores, 2))
print(' 5 folds 의 평균 RMSE : {0:.3f} '.format(avg_rmse))